In [14]:
from constants import *
from datasource import DATASOURCE
import os
from SPARQLWrapper import SPARQLWrapper, JSON, POST
import json 


DS = DATASOURCE # Alias

def new_sparqlwrapper(endpoint,user=None,password=None):
  sparql = SPARQLWrapper(endpoint)
  sparql.setReturnFormat(JSON)
  sparql.addCustomHttpHeader('Accept','application/sparql-results+json')
  sparql.addCustomHttpHeader('Content-Type','application/x-www-form-urlencoded; charset=UTF-8')
  sparql.setMethod(POST)
  if not user==None:
    sparql.setCredentials(user,password)
  return sparql

def execute_count_query(sparql, query):
  sparql.setQuery(query)
  result = sparql.query()
  count_dict = result.convert()
  count = None
  index = 0
  for result in count_dict["results"]["bindings"]: # Take the first entry of iterator; only one row
    if index>=1:
      raise Exception("Count queries are supposed to only have one row.")
    count = int(result['count']['value'])
    index += 1
  if count==None:
    raise Exception("No results returned from query:\n"+query)
  return count

def execute_query(sparql, query):
  sparql.setQuery(query)
  result = sparql.query().convert()
  return result["results"]["bindings"]

print("Init complete")


Init complete


In [15]:
COUNT_ALL_RESOLUTIONS = """\
{prefixes}
SELECT (COUNT(DISTINCT ?res) as ?count) WHERE {{
  ?res a besluit:Besluit.
}}
""".format(
  prefixes=PREFIXES_QUERY_PART
)

COUNT_LINKED_RESOLUTIONS = """\
{prefixes}
SELECT (COUNT(DISTINCT ?res) as ?count) WHERE {{
  ?administrativeUnit 
    a besluit:Bestuurseenheid;
    skos:prefLabel ?adminUnitLabel;
    org:classification <http://data.vlaanderen.be/id/concept/BestuurseenheidClassificatieCode/5ab0e9b8a3b2ca7c5e000001>.

  ?governingBodyAbstract
    a besluit:Bestuursorgaan;
    besluit:bestuurt ?administrativeUnit;
    org:classification <http://data.vlaanderen.be/id/concept/BestuursorgaanClassificatieCode/5ab0e9b8a3b2ca7c5e000005>.
  
  {{
    ?session 
      a besluit:Zitting;
      besluit:behandelt ?agendaItem;
      besluit:isGehoudenDoor ?governingBodyAbstract.
  }} UNION {{
    ?session 
      a besluit:Zitting;
      besluit:behandelt ?agendaItem;
      besluit:isGehoudenDoor ?governingBodyTimeSpecified.

    ?governingBodyTimeSpecified 
      a besluit:Bestuursorgaan;
      mandaat:isTijdspecialisatieVan ?governingBodyAbstract.
  }}
  ?agendaItem a besluit:Agendapunt .

  ?agendaItemHandling 
    a besluit:BehandelingVanAgendapunt;
    dct:subject ?agendaItem;
    prov:generated ?res.
  
  ?res a besluit:Besluit.
}}
""".format(
  prefixes=PREFIXES_QUERY_PART,
)

COUNT_LINKED_RESOLUTIONS_ALT = """\
{prefixes}
SELECT (COUNT(DISTINCT ?res) as ?count) WHERE {{
  ?administrativeUnit 
    a besluit:Bestuurseenheid;
    skos:prefLabel ?adminUnitLabel.

  ?governingBodyAbstract
    a besluit:Bestuursorgaan;
    besluit:bestuurt ?administrativeUnit.
  {{
    ?session 
      a besluit:Zitting;
      besluit:behandelt ?agendaItem;
      besluit:isGehoudenDoor ?governingBodyAbstract.
  }} UNION {{
    ?session 
      a besluit:Zitting;
      besluit:behandelt ?agendaItem;
      besluit:isGehoudenDoor ?governingBodyTimeSpecified.

    ?governingBodyTimeSpecified 
      a besluit:Bestuursorgaan;
      mandaat:isTijdspecialisatieVan ?governingBodyAbstract.
  }}
  ?agendaItem a besluit:Agendapunt .

  ?agendaItemHandling 
    a besluit:BehandelingVanAgendapunt;
    dct:subject ?agendaItem;
    prov:generated ?res.
  
  ?res a besluit:Besluit.
}}
""".format(
  prefixes=PREFIXES_QUERY_PART,
)

COUNT_CLASSIFIED_RESOLUTIONS = """\
{prefixes}
SELECT (COUNT(DISTINCT ?res) as ?count) WHERE {{
  ?res a besluit:Besluit, ?class.
  FILTER(REGEX(STR(?class),"^https:\\\\/\\\\/data\\\\.vlaanderen\\\\.be\\\\/id\\\\/concept\\\\/BesluitType\\\\/.*"))
}}
""".format(
  prefixes=PREFIXES_QUERY_PART,
)

COUNT_FISCAL_RESOLUTIONS = """\
{prefixes}
SELECT (COUNT(DISTINCT ?res) as ?count) WHERE {{
{fiscal_classes}
  ?res a besluit:Besluit, ?fiscalClass.
}}
""".format(
  prefixes=PREFIXES_QUERY_PART,
  fiscal_classes=FISCAL_RESOLUTUON_CLASSES_QUERY_PART,
)

COUNT_COMPLETE_FISCAL_RESOLUTIONS = """\
{prefixes}
SELECT (COUNT(DISTINCT ?res) as ?count) WHERE {{
{fiscal_classes}
  ?res a besluit:Besluit, ?fiscalClass.

  ?res 
    eli:description ?description;
    prov:value ?value;
    eli:title ?title;
    eli:language [ a skos:Concept ];
    eli:has_part [ a besluit:Artikel ].
  
  FILTER(
    datatype(?description) = xsd:string &&
    isLiteral(?value) &&
    datatype(?title) = xsd:string
  )
}}
""".format(
  prefixes=PREFIXES_QUERY_PART,
  fiscal_classes=FISCAL_RESOLUTUON_CLASSES_QUERY_PART,
)

COUNT_COMPLETE_FISCAL_RESOLUTIONS_ALT = """\
{prefixes}
SELECT (COUNT(DISTINCT ?res) as ?count) WHERE {{
{fiscal_classes}
  ?res a besluit:Besluit, ?fiscalClass.

  ?res 
    eli:description ?description;
    prov:value ?value;
    eli:title ?title.
  
  FILTER(
    datatype(?description) = xsd:string &&
    isLiteral(?value) &&
    datatype(?title) = xsd:string
  )
}}
""".format(
  prefixes=PREFIXES_QUERY_PART,
  fiscal_classes=FISCAL_RESOLUTUON_CLASSES_QUERY_PART,
)

COUNT_LINKED_COMPLETE_FISCAL_RESOLUTIONS = """\
{prefixes}
SELECT (COUNT(DISTINCT ?res) as ?count) WHERE {{
{fiscal_classes}
  ?administrativeUnit 
    a besluit:Bestuurseenheid;
    skos:prefLabel ?adminUnitLabel;
    org:classification <http://data.vlaanderen.be/id/concept/BestuurseenheidClassificatieCode/5ab0e9b8a3b2ca7c5e000001>.

  ?governingBodyAbstract
    a besluit:Bestuursorgaan;
    besluit:bestuurt ?administrativeUnit;
    org:classification <http://data.vlaanderen.be/id/concept/BestuursorgaanClassificatieCode/5ab0e9b8a3b2ca7c5e000005>.
  
  {{
    ?session 
      a besluit:Zitting;
      besluit:behandelt ?agendaItem;
      besluit:isGehoudenDoor ?governingBodyAbstract.
  }} UNION {{
    ?session 
      a besluit:Zitting;
      besluit:behandelt ?agendaItem;
      besluit:isGehoudenDoor ?governingBodyTimeSpecified.

    ?governingBodyTimeSpecified 
      a besluit:Bestuursorgaan;
      mandaat:isTijdspecialisatieVan ?governingBodyAbstract.
  }}
  ?agendaItem a besluit:Agendapunt .

  ?agendaItemHandling 
    a besluit:BehandelingVanAgendapunt;
    dct:subject ?agendaItem;
    prov:generated ?res.

  ?res a besluit:Besluit, ?fiscalClass.

  ?res 
    eli:description ?description;
    prov:value ?value;
    eli:title ?title;
    eli:language [ a skos:Concept ];
    eli:has_part [ a besluit:Artikel ].
  
  FILTER(
    datatype(?description) = xsd:string &&
    isLiteral(?value) &&
    datatype(?title) = xsd:string
  )
}}
""".format(
  prefixes=PREFIXES_QUERY_PART,
  fiscal_classes=FISCAL_RESOLUTUON_CLASSES_QUERY_PART,
)

COUNT_LINKED_COMPLETE_FISCAL_RESOLUTIONS_ALT = """\
{prefixes}
SELECT (COUNT(DISTINCT ?res) as ?count) WHERE {{
{fiscal_classes}
  ?administrativeUnit 
    a besluit:Bestuurseenheid;
    skos:prefLabel ?adminUnitLabel;
    org:classification <http://data.vlaanderen.be/id/concept/BestuurseenheidClassificatieCode/5ab0e9b8a3b2ca7c5e000001>.

  ?governingBodyAbstract
    a besluit:Bestuursorgaan;
    besluit:bestuurt ?administrativeUnit;
    org:classification <http://data.vlaanderen.be/id/concept/BestuursorgaanClassificatieCode/5ab0e9b8a3b2ca7c5e000005>.
  
  {{
    ?session 
      a besluit:Zitting;
      besluit:behandelt ?agendaItem;
      besluit:isGehoudenDoor ?governingBodyAbstract.
  }} UNION {{
    ?session 
      a besluit:Zitting;
      besluit:behandelt ?agendaItem;
      besluit:isGehoudenDoor ?governingBodyTimeSpecified.

    ?governingBodyTimeSpecified 
      a besluit:Bestuursorgaan;
      mandaat:isTijdspecialisatieVan ?governingBodyAbstract.
  }}
  ?agendaItem a besluit:Agendapunt .

  ?agendaItemHandling 
    a besluit:BehandelingVanAgendapunt;
    dct:subject ?agendaItem;
    prov:generated ?res.

  ?res a besluit:Besluit, ?fiscalClass.

  ?res 
    eli:description ?description;
    prov:value ?value;
    eli:title ?title.
  
  FILTER(
    datatype(?description) = xsd:string &&
    isLiteral(?value) &&
    datatype(?title) = xsd:string
  )
}}
""".format(
  prefixes=PREFIXES_QUERY_PART,
  fiscal_classes=FISCAL_RESOLUTUON_CLASSES_QUERY_PART,
)

print("Queries loaded")


Queries loaded


In [16]:

import copy 
import re
# [Identifier, query string van query met enkel ?count, enabled]
QUERY_JOBS = [
  ["All",COUNT_ALL_RESOLUTIONS,False],
  ["Linked",COUNT_LINKED_RESOLUTIONS,False],
  ["Linked-alt",COUNT_LINKED_RESOLUTIONS_ALT,False],
  ["Classified",COUNT_CLASSIFIED_RESOLUTIONS,False],
  ["Fiscal",COUNT_FISCAL_RESOLUTIONS,False],
  ["Fiscal-complete",COUNT_COMPLETE_FISCAL_RESOLUTIONS,True],
  ["Fiscal-complete-alt",COUNT_COMPLETE_FISCAL_RESOLUTIONS_ALT,False],
  ["Fiscal-linked-complete",COUNT_LINKED_COMPLETE_FISCAL_RESOLUTIONS,True],
  ["Fiscal-linked-complete-alt",COUNT_LINKED_COMPLETE_FISCAL_RESOLUTIONS_ALT,False],
]

sparql = new_sparqlwrapper(DS["endpoint"],DS["user"],DS["password"])

output = {}
exceptions = {}

for job in QUERY_JOBS:
  if not job[2]:
    continue
  print("Querying '{}'".format(job[0]))
  try:
    count = execute_count_query(sparql,job[1])
    output[job[0]] = count
    print("Query '{}' succeeded".format(job[0]))
  except Exception as e:
    exceptions[job[0]] = [job[1],e]
    print("Query '{}' failed. See log".format(job[0]))

output_filename = os.getcwd() + "/output.json"

try:
  with open(output_filename,"r",encoding="utf-8") as file:
    previous_run = json.load(file)
  output = {**previous_run, **output}
except:
  print("No file found from previous run.")

json_output = json.dumps(output,indent=4)
print('Results of the current count aggregated with the results of the previous run:')
print(json_output)
with open(output_filename,"w",encoding="utf-8") as file:
  file.write(json_output)

lineNrs = False
if len(exceptions) > 0:
  print("Some of the queries failed. Please see query-errors.log for more info.")
  with open(os.getcwd() + "/query-errors.log","w",encoding="utf-8") as file:
    for key in exceptions:
      file.write("Exception occurred in query:\n")
      query = exceptions[key][0]
      if lineNrs:
        linenr = 1
        for line in query.split("\n"):
          file.write("{}: {}\n".format(linenr,line))
          linenr += 1
      else:
        file.write(query)
        file.write("\n")
      file.write("---\n")
      file.write("Stacktrace\n")
      file.write(str(exceptions[key][1]))
      file.write("---\n")



Querying 'Fiscal-complete'
Query 'Fiscal-complete' succeeded
Querying 'Fiscal-linked-complete'
Query 'Fiscal-linked-complete' succeeded
Results of the current count aggregated with the results of the previous run:
{
    "All": 2434838,
    "Linked": 301471,
    "Linked-alt": 1706701,
    "Classified": 77737,
    "Fiscal": 2067,
    "Fiscal-complete": 546,
    "Fiscal-complete-alt": 1206,
    "Fiscal-linked-complete": 483,
    "Fiscal-linked-complete-alt": 1107
}


In [11]:
QUERY_FISCAL_PER_MUNICIPALITY = """\
{prefixes}
SELECT ?municipalityName (COUNT(DISTINCT ?res) as ?count) WHERE {{
  {municipalities}
  {fiscal_classes}
  ?administrativeUnit 
    a besluit:Bestuurseenheid;
    skos:prefLabel ?adminUnitLabel;
    org:classification <http://data.vlaanderen.be/id/concept/BestuurseenheidClassificatieCode/5ab0e9b8a3b2ca7c5e000001>.

  ?governingBodyAbstract
    a besluit:Bestuursorgaan;
    besluit:bestuurt ?administrativeUnit;
    org:classification <http://data.vlaanderen.be/id/concept/BestuursorgaanClassificatieCode/5ab0e9b8a3b2ca7c5e000005>.
  {{
    ?session 
      a besluit:Zitting;
      besluit:behandelt ?agendaItem;
      besluit:isGehoudenDoor ?governingBodyAbstract.
  }} UNION {{
    ?session 
      a besluit:Zitting;
      besluit:behandelt ?agendaItem;
      besluit:isGehoudenDoor ?governingBodyTimeSpecified.

    ?governingBodyTimeSpecified 
      a besluit:Bestuursorgaan;
      mandaat:isTijdspecialisatieVan ?governingBodyAbstract.
  }}
  ?agendaItem a besluit:Agendapunt .

  ?agendaItemHandling 
    a besluit:BehandelingVanAgendapunt;
    dct:subject ?agendaItem;
    prov:generated ?res.
  
  ?res a besluit:Besluit, ?fiscalClass.

  ?res 
    eli:description ?description;
    prov:value ?value;
    eli:title ?title.
  
  FILTER(
    datatype(?description) = xsd:string &&
    isLiteral(?value) &&
    datatype(?title) = xsd:string
  )

  BIND(REPLACE(?adminUnitLabel, "^\\\\s*Gemeente\\\\s+([\\\\w-\\\\.]+)\\\\s*$","$1") as ?fixed_adminUnitLabel)
  FILTER(?fixed_adminUnitLabel = ?municipalityName)
}}
GROUP BY ?municipalityName
""".format(
  prefixes=PREFIXES_QUERY_PART,
  fiscal_classes=FISCAL_RESOLUTUON_CLASSES_QUERY_PART,
  municipalities=MUNICIPALITY_SUPPLIER_VALUES_QUERY_PART,
)

QUERY_FISCAL_PER_MUNICIPALITY_ALT = """\
{prefixes}
SELECT ?municipalityName (COUNT(DISTINCT ?res) as ?count) WHERE {{
  {municipalities}
  {fiscal_classes}
  ?administrativeUnit 
    a besluit:Bestuurseenheid;
    skos:prefLabel ?adminUnitLabel;
    org:classification <http://data.vlaanderen.be/id/concept/BestuurseenheidClassificatieCode/5ab0e9b8a3b2ca7c5e000001>.

  ?governingBodyAbstract
    a besluit:Bestuursorgaan;
    besluit:bestuurt ?administrativeUnit;
    org:classification <http://data.vlaanderen.be/id/concept/BestuursorgaanClassificatieCode/5ab0e9b8a3b2ca7c5e000005>.
  {{
    ?session 
      a besluit:Zitting;
      besluit:behandelt ?agendaItem;
      besluit:isGehoudenDoor ?governingBodyAbstract.
  }} UNION {{
    ?session 
      a besluit:Zitting;
      besluit:behandelt ?agendaItem;
      besluit:isGehoudenDoor ?governingBodyTimeSpecified.

    ?governingBodyTimeSpecified 
      a besluit:Bestuursorgaan;
      mandaat:isTijdspecialisatieVan ?governingBodyAbstract.
  }}
  ?agendaItem a besluit:Agendapunt .

  ?agendaItemHandling 
    a besluit:BehandelingVanAgendapunt;
    dct:subject ?agendaItem;
    prov:generated ?res.
  
  ?res a besluit:Besluit, ?fiscalClass.

  ?res 
    eli:description ?description;
    prov:value ?value;
    eli:title ?title.
  
  FILTER(
    datatype(?description) = xsd:string &&
    isLiteral(?value) &&
    datatype(?title) = xsd:string
  )

  BIND(REPLACE(?adminUnitLabel, "^\\\\s*Gemeente\\\\s+([\\\\w-\\\\.]+)\\\\s*$","$1") as ?fixed_adminUnitLabel)
  FILTER(?fixed_adminUnitLabel = ?municipalityName)
}}
GROUP BY ?municipalityName
""".format(
  prefixes=PREFIXES_QUERY_PART,
  fiscal_classes=FISCAL_RESOLUTUON_CLASSES_QUERY_PART,
  municipalities=MUNICIPALITY_SUPPLIER_VALUES_QUERY_PART,
)


In [12]:
# sparql = new_sparqlwrapper(DS["endpoint"])
print("Querying for municipality distribution")
result = execute_query(sparql,QUERY_FISCAL_PER_MUNICIPALITY_ALT) # Change to alt if necassary
print("Query succeeded")
output = {}
for row in result:
  output[row['municipalityName']['value']] = row['count']['value']

Querying for municipality distribution
Query succeeded


In [13]:
# Generate HTML
LIST_TEMPLATE = """\

<ul class="flex-list-predicates">
{items}
</ul>

"""

ITEM_TEMPLATE = """\
<li>{municipality}: {count}</li>
"""

items = []
counter = 0
total_res = 0
for key in sorted(output.keys()):
  counter += 1
  total_res += int(output[key])
  municipality = key if len(key) < 17 else key[0:15] + "&hellip;"
  
  items.append(ITEM_TEMPLATE.format(
    municipality=municipality,
    count=output[key]
  ))

print("Added {} rows. Total {} resolutuins".format(counter, total_res))
table = LIST_TEMPLATE.format(items="".join(items).strip())

with open(os.getcwd() + "/municipality-count-table.md","w", encoding="utf-8") as file:
  file.write(table)

print("Output generated in municipality-count-table.md")

Added 115 rows. Total 1104 resolutuins
Output generated in municipality-count-table.md
